In [36]:
import jax
import jax.numpy as jnp
import pandas as pd
max_horizon = 36
jax.config.update("jax_enable_x64", True)
lst_K = [1, 3, 5, 10, 15, 20, 25]
dataset = 'scs'
gamma = -1
lag = 36

In [37]:
tensor_dir_in = f'/home/james/projects/tsfc/code/code_11092024/results_oos/multiperiod/{dataset}/tensor_fig_oos_ret_rankptf_ver3/'
dict_tensor_oos = jnp.load(tensor_dir_in + 'dict_tensor_oos_1982.pkl', allow_pickle=True)
pca_dir_in = f'/home/james/projects/tsfc/code/code_11092024/results_oos/multiperiod/{dataset}/pca_fig_oos_ret_rankptf_ver3/'
dict_pca_oos = jnp.load(pca_dir_in + 'dict_pca_oos_1982.pkl', allow_pickle=True)

sr_lst = []
for horizon in range(1, max_horizon + 1):
    sr = dict_pca_oos[gamma][horizon].mean(axis=1) / dict_pca_oos[gamma][horizon].std(axis=1)
    sr_lst.append(jnp.expand_dims(sr, axis=0))

sr_pca = jnp.concatenate(sr_lst, axis=0) # dim: (max_horizon, len(lst_K))
sr_tensor = dict_tensor_oos["TFM"][lag].mean(axis=0) / dict_tensor_oos["TFM"][lag].std(axis=0)

In [41]:
x = sr_pca[:, 0]
y = sr_tensor[:, 0]
jnp.mean(y - x), lst_K[0]

(Array(0.11038117, dtype=float64), 1)

In [78]:
y = sr_pca[:, 0]

y * jnp.sqrt(1 / jnp.arange(1, 37)) * jnp.sqrt(12)

Array([0.4193373 , 0.37112065, 0.32629429, 0.2969318 , 0.28099487,
       0.25949437, 0.13386099, 0.07585626, 0.07295683, 0.07940755,
       0.09878111, 0.11818797, 0.14894346, 0.17217478, 0.1895464 ,
       0.20583558, 0.22186856, 0.22923145, 0.23475323, 0.2387511 ,
       0.2470701 , 0.25523704, 0.25887307, 0.26392608, 0.26916663,
       0.27062835, 0.27147279, 0.27107705, 0.27201321, 0.27180501,
       0.27007151, 0.26777731, 0.26089671, 0.25390391, 0.24504195,
       0.23693364], dtype=float64)

In [69]:
temp = dict_tensor_oos["TFM"][36][:, :, 6]
(temp.mean(axis=0) / temp.std(axis=0)) / jnp.ar

Array([0.50786427, 0.60541793, 0.7121959 , 0.81854574, 0.86131744,
       0.89949768, 0.93202775, 0.96784028, 0.99853802, 1.01859176,
       1.04232528, 1.04251084, 1.05382212, 1.06132026, 1.05306033,
       1.0443778 , 1.03541013, 1.02813382, 1.02820339, 1.0320684 ,
       1.04121321, 1.04142217, 1.04882719, 1.05428774, 1.06146839,
       1.06858731, 1.07421879, 1.07242219, 1.07872479, 1.07799458,
       1.08939502, 1.08986177, 1.09392505, 1.09851547, 1.1089901 ,
       1.11277326], dtype=float64)

In [4]:
(y[-1] - x[-1]) / x[-1]

Array(2.34636166, dtype=float64)

In [33]:
def mean_log_sharpe_diff(dataset: str, gamma: int, lag: int, percentage: bool = False):
    tensor_dir_in = f'/home/james/projects/tsfc/code/code_11092024/results_oos/multiperiod/{dataset}/tensor_fig_oos_ret_rankptf_ver3/'
    dict_tensor_oos = jnp.load(tensor_dir_in + 'dict_tensor_oos_1982.pkl', allow_pickle=True)
    pca_dir_in = f'/home/james/projects/tsfc/code/code_11092024/results_oos/multiperiod/{dataset}/pca_fig_oos_ret_rankptf_ver3/'
    dict_pca_oos = jnp.load(pca_dir_in + 'dict_pca_oos_1982.pkl', allow_pickle=True)
    
    sr_lst = []
    for horizon in range(1, max_horizon + 1):
        sr = dict_pca_oos[gamma][horizon].mean(axis=1) / dict_pca_oos[gamma][horizon].std(axis=1)
        sr_lst.append(jnp.expand_dims(sr, axis=0))

    sr_pca = jnp.concatenate(sr_lst, axis=0) # dim: (max_horizon, len(lst_K))
    sr_tensor = dict_tensor_oos["TFM"][lag].mean(axis=0) / dict_tensor_oos["TFM"][lag].std(axis=0)
    if percentage:
        data = jnp.mean((sr_tensor - sr_pca) / sr_pca, axis=0)
    else:
        data = jnp.mean(sr_tensor - sr_pca, axis=0)

    df = pd.DataFrame(data)
    df.columns = [f'{dataset}, Gamma={gamma}, Lag={lag}']  # Update column name
    df.index = lst_K 
    return df

In [35]:
mean_log_sharpe_diff('scs', -1, 36)

,"scs, Gamma=-1, Lag=36"
1,0.110381
3,0.067687
5,0.273864
10,0.512959
15,0.567720
20,0.584478
25,0.478860


In [29]:
mean_log_sharpe_diff('scs', 10, 36, True)

,"scs, Gamma=10, Lag=36"
1,0.157782
3,0.252800
5,0.322152
10,0.737776
15,0.923209
20,0.728615
25,0.603557
